# Customer Attrition Model - Additional Regression Model
## Regression Analysis on Total Revenue

## 6.1 Notebook Brief

The notebook is a side analysis on the total revenue chargess from the dataset. This is to understand the factors affecting the revuenue and charges applied to the customer and has no affect on the classification prediction. 

## 6.2 Data Imports

In [2]:
#import library
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#import LR 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
#read csv
telco_final = pd.read_csv('../data/telco_cleaned_final.csv')

#set display max 100
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [4]:
telco_final.head()

,gender,age,under_30,senior_citizen,married,dependents,number_of_dependents,city,zip_code,latitude,longitude,tenure_months,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,churn_score,cltv,churn_reason,referred_a_friend,number_of_referrals,offer,avg_monthly_long_distance_charges,internet_type,avg_monthly_gb_download,streaming_music,unlimited_data,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,satisfaction_score,customer_status,churn_category,pop_density,new_user
0,0,78,0,1,0,0,0,Los Angeles,90022,34.023810,-118.156582,1,0,0,DSL,0,0,1,0,0,1,Month-to-month,1,Electronic check,100,5433,Competitor offered more data,0,0,None,0.00,DSL,8,0,0,39.65,39.65,0.00,20,0.00,59.65,3,1,Competitor,15334,0
1,1,74,0,1,1,1,1,Los Angeles,90063,34.044271,-118.185237,8,1,1,Fiber optic,0,1,0,0,0,0,Month-to-month,1,Credit card (automatic),69,5302,Competitor made better offer,1,1,Offer E,48.85,Fiber Optic,17,0,1,80.65,633.30,0.00,0,390.80,1024.10,3,1,Competitor,17038,0
2,0,71,0,1,0,1,3,Los Angeles,90065,34.108833,-118.229715,18,1,1,Fiber optic,0,0,0,0,1,1,Month-to-month,1,Electronic check,81,3179,Competitor made better offer,0,0,Offer D,11.33,Fiber Optic,52,1,1,95.45,1752.55,45.61,0,203.94,1910.88,2,1,Competitor,8625,0
3,1,78,0,1,1,1,1,Inglewood,90303,33.936291,-118.332639,25,1,0,Fiber optic,0,1,1,0,1,1,Month-to-month,1,Electronic check,88,5337,Limited range of services,1,1,Offer C,19.76,Fiber Optic,12,0,1,98.50,2514.50,13.43,0,494.00,2995.07,2,1,Dissatisfaction,11799,0
4,1,80,0,1,1,1,1,Whittier,90602,33.972119,-118.020188,37,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,67,2793,Extra data charges,1,1,Offer C,6.33,Fiber Optic,14,0,1,76.50,2868.15,0.00,0,234.21,3102.36,2,1,Price,6947,0


##  6.3 Total Revenue Breakdown

In [6]:
#drop categorical and churn features as it does not affect the charges apply 
telco_final = telco_final.drop(columns=['city', 'zip_code', 'latitude', 'longitude',
                                      'churn_score','churn_reason','tenure_months',
                                      'avg_monthly_long_distance_charges', 'internet_type',
                                      'avg_monthly_gb_download','churn_category','customer_status',
                                      'satisfaction_score', 'cltv'])

### 6.3.1 One Hot Encoding for Services

Main telco services such as `internet_services`, `contract`, `payment_method` and `offer` are one hot encoded to see the effect on each categorical features on the price.

In [7]:
#ohe above
ts_dummies = pd.get_dummies(data=telco_final, drop_first=True)

### 6.3.2 Linear Regression Modeling

In [8]:
#shape check 5174 row x 44 cols
ts_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 40 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender                                  7043 non-null   int64  
 1   age                                     7043 non-null   int64  
 2   under_30                                7043 non-null   int64  
 3   senior_citizen                          7043 non-null   int64  
 4   married                                 7043 non-null   int64  
 5   dependents                              7043 non-null   int64  
 6   number_of_dependents                    7043 non-null   int64  
 7   phone_service                           7043 non-null   int64  
 8   multiple_lines                          7043 non-null   int64  
 9   online_security                         7043 non-null   int64  
 10  online_backup                           7043 non-null   int6

In [9]:
#define X and y 
X = ts_dummies.drop(columns='total_revenue')
y = ts_dummies.total_revenue

In [10]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [11]:
#instantiate LR
lr = LinearRegression()
lr.fit(X_train, y_train)
print(f'LR Train Score: {lr.score(X_train, y_train)}')
print(f'LR Test Score: {lr.score(X_test, y_test)}')

LR Train Score: 1.0
LR Test Score: 1.0


In [12]:
coefs = pd.Series(data=lr.coef_, index=X_train.columns).round()
coefs = coefs[coefs!=0]
coefs

total_charges                  1.0
total_refunds                 -1.0
total_extra_data_charges       1.0
total_long_distance_charges    1.0
dtype: float64

In [13]:
#sanity check 
telco_final['total_revenue_compare'] = telco_final['total_charges'] + telco_final['total_long_distance_charges'] + telco_final['total_extra_data_charges'] - telco_final['total_refunds']
telco_final[['total_charges', 'total_revenue', 'total_revenue_compare']].head(10)

,total_charges,total_revenue,total_revenue_compare
0,39.65,59.65,59.65
1,633.30,1024.10,1024.10
2,1752.55,1910.88,1910.88
3,2514.50,2995.07,2995.07
4,2868.15,3102.36,3102.36
5,2135.50,2235.41,2235.41
6,70.45,85.73,85.73
7,2651.20,2610.25,2610.25
8,1145.70,1806.75,1806.75
9,503.60,681.20,681.20


In [14]:
#error check
mean_absolute_error(telco_final.total_revenue, telco_final.total_revenue_compare).round()

0.0

From a quick analysis above we can associate the `total_revenue` drive comes from mainly the summation of `total_charges`, `total_extra_data_charges`, `total_long_distance_charges` including the `total_refunds` deduction.

## 6.4 Total Charges Breakdown 

From the above, we know that the total charges are also calculated with effects from the offers and other services customized by customers. In this section, we will attempt to identify the factors affecting the total charges.

In [15]:
#define X and y 
X = ts_dummies.drop(columns=['total_revenue','total_charges','total_refunds','total_long_distance_charges','total_extra_data_charges'])
y = ts_dummies.total_charges

In [16]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [17]:
#instantiate LR
lr2 = LinearRegression()
lr2.fit(X_train, y_train)
print(f'LR Train Score: {lr2.score(X_train, y_train):.2f}')
print(f'LR Test Score: {lr2.score(X_test, y_test):.2f}')

LR Train Score: 0.79
LR Test Score: 0.79


In [18]:
### identify coef
coefs = pd.Series(data=lr2.coef_, index=X.columns).astype(int)
coefs = coefs[coefs!=0]
coefs.sort_values(ascending=False)

offer_Offer A                             1291
contract_Two year                         1120
internet_service_No                        965
contract_One year                          817
offer_Offer B                              715
online_backup                              568
married                                    471
online_security                            369
multiple_lines                             366
device_protection                          345
senior_citizen                             230
streaming_movies                           201
tech_support                               181
streaming_tv                                86
monthly_charge                              45
paperless_billing                           32
number_of_referrals                         17
payment_method_Credit card (automatic)      10
age                                         -3
dependents                                  -7
number_of_dependents                       -14
gender       

From the above coeffcient, we can conclude that Offer A has the highest impact on the overall `total_charges` at a whopping \\$1,291 and alternatively for Offer D and E with a discounted charges at \\$ 566 and \\$ 725 respectively. 

Contracts at minimum 2 years has also impact on the overall `total_charges` where we see \\$ 1,120 more charges compared to only \\$ 817 with one year less commitment.

Most add-ons services such as online security or device protection has a positive impact to the price point except for streaming_music, unlimited_data and phone_service. As phone calls are the main service a telco company is providing, the cost of an additional communication port has its fair share of minimum cost. This stands true for customer acquisition cost of \\$105, in which the goal is to attempt for customer retention plan that cost lower than this. 

As for streaming_music and unlimited data, we expect the same due to the base cost that is higher. The possibility of increasing the charges for these two respectively are to be investigated further on it potential cost that may led to losing the existing customer base. 